In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
from dariah_topics import evaluation

/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationW

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

['corpus_txt/Poe_ThePurloinedLetter.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Kipling_ThyServantaDog.txt',
 'corpus_txt/Howard_ShadowsintheMoonlight.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

['corpus_csv/Howard_ShadowsintheMoonlight.txt.csv',
 'corpus_csv/Lovecraft_AttheMountainofMadness.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Doyle_AStudyinScarlet.txt.csv',
 'corpus_csv/Poe_TheCaskofAmontillado.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt/Poe_ThePurloinedLetter.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Kipling_ThyServantaDog.txt',
 'corpus_txt/Howard_ShadowsintheMoonlight.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[0][:5]

['the', 'purloined', 'letter', 'edgar', 'allan']

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow[:5]

0
doc_id token_id   
1      19231     1
       12292     5
       16389     1
       6         1
       4109      3

#### Convert to Doc-Topic Matrix

In [11]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [12]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "en.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [13]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [14]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

53624
48323


##### Create Feature Remove Lists

In [15]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

100

In [16]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

19232

##### Remove Features

In [17]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

In [18]:
len(clean_term_frequency)

4587

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [19]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

In [20]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [21]:
mm = MmCorpus("gb_plain.mm")

In [22]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [23]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [24]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [25]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [26]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [27]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id)

WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [28]:
topic_nr_x = model.get_topic_terms(10)

[type2id[i[0]] for i in topic_nr_x]

['torpedo',
 'belief:—and',
 'head-on-paws',
 'wiping',
 'professor',
 'old-fashioned',
 'dark--the',
 'a-leanin',
 'casements',
 'calculated']

In [29]:
topics = model.show_topics(num_topics = 60)

In [30]:
topics[:2]

[(99,
  '0.044*"torpedo" + 0.024*"belief:—and" + 0.020*"head-on-paws" + 0.014*"professor" + 0.013*"wiping" + 0.012*"dark--the" + 0.010*"casements" + 0.009*"site" + 0.009*"a-leanin" + 0.008*"shift"'),
 (28,
  '0.039*"torpedo" + 0.033*"belief:—and" + 0.026*"head-on-paws" + 0.018*"wiping" + 0.015*"a-leanin" + 0.015*"professor" + 0.014*"dark--the" + 0.013*"site" + 0.012*"old-fashioned" + 0.012*"calculated"')]

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [31]:
"""
import os

#Variablen zum Testen
#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
#malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)

path_to_corpus = os.path.join(os.path.abspath('.'), 'corpus_txt')
assert os.path.exists(path_to_corpus)

malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)
"""

'\nimport os\n\n#Variablen zum Testen\n#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"\n#malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)\n\npath_to_corpus = os.path.join(os.path.abspath(\'.\'), \'corpus_txt\')\nassert os.path.exists(path_to_corpus)\n\nmalletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)\n'

##### Mallet output erzeugen

In [32]:
"""outfolder = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
mal.create_mallet_model(malletBinary,outfolder)"""

'outfolder = os.path.join(os.path.abspath(\'.\'), "tutorial_supplementals/mallet_output")\nmal.create_mallet_model(malletBinary,outfolder)'

##### Matrix von Mallet output erzeugen

In [33]:
"""basepath = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
doc_topics = os.path.join(basepath, "doc_topics.txt")"""

'basepath = os.path.join(os.path.abspath(\'.\'), "tutorial_supplementals/mallet_output")\ndoc_topics = os.path.join(basepath, "doc_topics.txt")'

In [34]:
"""mal.create_MalletMatrix(doc_topics)"""

'mal.create_MalletMatrix(doc_topics)'

## Visualisierung 

In [35]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

"lda_model = 'out_easy/corpus.lda'\ncorpus = 'out_easy/corpus.mm'\ndictionary = 'out_easy/corpus.dict'\ndoc_labels = 'out_easy/corpus_doclabels.txt'\ninteractive  = False\n\nvis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)"

#### Heatmap

In [36]:
#heatmap = visual.make_heatmap()

In [37]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [38]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [39]:
#vis.make_interactive()

In [40]:
#vis.save_interactive("./visualizations/interactive")

## Evaluation

In [41]:
segmented_topics = evaluation.topic_segmenter(topics)
segmented_topics[0][:5]

[('torpedo', 'belief:—and'),
 ('torpedo', 'head-on-paws'),
 ('torpedo', 'professor'),
 ('torpedo', 'wiping'),
 ('torpedo', 'dark--the')]

In [42]:
umass_probability = evaluation.token_probability(doc_tokens, segmented_topics)
umass_probability

torpedo                    {2}
calculated       {8, 9, 10, 4}
clean-shaven        {1, 11, 9}
site                    {8, 1}
wiping               {8, 9, 4}
puppies                {2, 12}
dark--the                 {12}
head-on-paws               {2}
a-leanin                   {9}
shift                {8, 9, 4}
professor           {8, 9, 10}
old-fashioned          {1, 11}
hart's-tongue             {11}
casements              {3, 13}
belief:—and               {16}
dtype: object

In [43]:
umass = evaluation.calculate_umass(segmented_topics, umass_probability, doc_tokens, 60)
umass

0.000682097914687714

In [44]:
wiki_list = evaluation.wikipedia_table_crawler()

/usr/lib/python3/dist-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


In [45]:
wiki_corpus = evaluation.wikipedia_crawler(wiki_list, size=5)

In [46]:
uci_probability = evaluation.token_probability(wiki_corpus, segmented_topics)

In [47]:
pmi = evaluation.calculate_pointwise_mutual_information(segmented_topics, wiki_corpus, uci_probability)

In [48]:
uci = evaluation.calculate_uci(pmi, wiki_corpus, 60)
uci

0.0

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)